In [1]:
import os, sys, email, datetime, pprint, re, time, html, pickle
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *

pd.options.display.max_colwidth = 1000

In [34]:
with open('/Users/taroaso/myprojects/OpenIE/trec/output/tagme.pickle', mode="rb") as f:
    tagme_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/ner.pickle', mode="rb") as f:
     ner_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/type_translation.pickle', mode="rb") as f:
     translation = pickle.load(f)

In [3]:
from entityLinking import getWikidataID

In [4]:
for result in tagme_result["003b01c10c55$f0f62060$f3468ed5@huftis"]['annotations']:
    try:
        if result['rho'] >= 0.3:
            res = getWikidataID(result['id'])
            pprint.pprint(result)
            pprint.pprint(res)
    except:
        pass

{'end': 268,
 'id': 839715,
 'link_probability': 1,
 'rho': 0.7779867649078369,
 'spot': 'numeric entity',
 'start': 254,
 'title': 'Numeric character reference'}
{'batchcomplete': '',
 'query': {'pages': {'839715': {'ns': 0,
                                'pageid': 839715,
                                'pageprops': {'defaultsort': 'Numeric '
                                                             'Character '
                                                             'Reference',
                                              'wikibase_item': 'Q1204079'},
                                'title': 'Numeric character reference'}}}}
{'end': 287,
 'id': 59126,
 'link_probability': 0.37846481800079346,
 'rho': 0.3699946403503418,
 'spot': 'apostrophe',
 'start': 277,
 'title': 'Apostrophe'}
{'batchcomplete': '',
 'query': {'pages': {'59126': {'ns': 0,
                               'pageid': 59126,
                               'pageprops': {'wikibase-shortdesc': 'Punctuation '
  

In [58]:
ner_result[uid]

[{'spot': 'doc', 'start': 21, 'end': 24, 'type': 'ORG'},
 {'spot': '<', 'start': 109, 'end': 110, 'type': 'NORP'},
 {'spot': 'xhtml', 'start': 177, 'end': 182, 'type': 'ORG'},
 {'spot': 'xhtml', 'start': 237, 'end': 242, 'type': 'ORG'},
 {'spot': 'xhtml', 'start': 342, 'end': 347, 'type': 'ORG'},
 {'spot': 'javascript', 'start': 352, 'end': 362, 'type': 'PERSON'},
 {'spot': 'two', 'start': 427, 'end': 430, 'type': 'CARDINAL'},
 {'spot': 'john russell ve3ll@rac.ca',
  'start': 439,
  'end': 464,
  'type': 'PERSON'},
 {'spot': "L's", 'start': 476, 'end': 479, 'type': 'ORG'}]

In [8]:
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
BASE = Namespace("http://www.kde.cs.tsukuba.ac.jp/~aso/trec#")
#EMAIL = Namespace("http://www.w3.org/2000/10/swap/pim/email#")
OLIA = Namespace("http://purl.org/olia/olia.owl#")
NERD = Namespace("http://nerd.eurecom.fr/ontology#")
NIF = Namespace("http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#")
ITSRDF = Namespace("https://www.w3.org/2005/11/its/rdf#")
WD = Namespace("http://www.wikidata.org/entity/")

In [76]:
# RDFグラフ
g = Graph()
g.bind('',BASE)
g.bind('schema',SDO)
g.bind('foaf',FOAF)
g.bind('owl',OWL)
g.bind('rdf',RDF)
g.bind('rdfs',RDFS)
g.bind('olia',OLIA)
g.bind('nerd',NERD)
g.bind('nif',NIF)
g.bind('itsrdf',ITSRDF)
g.bind('wd',WD)

In [77]:
result = list(tagme_result.items())

In [78]:
uid = result[1][0]

##URI参照されるリソースは表記を変換する
emailmessage = re.sub(r'[^a-zA-Z_0-9]','_',uid) 

for annotation in result[1][1]['annotations']:
    if annotation['link_probability'] >= 0.3:
        spot = annotation['spot']
        start = annotation['start']
        end = annotation['end']
        offset_based_string = emailmessage + '#offset_' + str(start) + '_' + str(end)
        # Entity, Mentionに共通する記述
        g.add((BASE[emailmessage],SDO.mentions,BASE[offset_based_string]))
        g.add((BASE[offset_based_string],RDF.type,NIF.String))
        g.add((BASE[offset_based_string],NIF.beginIndex,Literal(start,datatype=XSD.integer)))
        g.add((BASE[offset_based_string],NIF.endIndex,Literal(end,datatype=XSD.integer)))
        g.add((BASE[offset_based_string],NIF.isString,Literal(spot,lang='en')))
        # Entity
        if 'rho' in annotation:
            rho = annotation['rho']
            pid = annotation['id']
            # 閾値0.3以上のEntityを記述する
            if annotation['rho'] >= 0.3:
                res = getWikidataID(pid)
                wid = res['query']['pages'][str(pid)]['pageprops']['wikibase_item']
                g.add((BASE[offset_based_string],ITSRDF.taIdentRef,WD[wid]))
                g.add((BASE[offset_based_string],ITSRDF.taConfidence,Literal(rho,datatype=XSD.decimal)))
            # 閾値0.3未満のEntityはMentionと同じ記述
            else:
                link_probability = annotation['link_probability']
                g.add((BASE[offset_based_string],RDF.type,NIF.EntityOccurrence))
                g.add((BASE[offset_based_string],NIF.entityOccurrenceConf,Literal(link_probability,datatype=XSD.decimal)))
                for ner in ner_result[uid]:
                    if ner['spot'] == spot:
                        spacy_type = ner['type']
                        schema_type = translation[spacy_type]
                        g.add((BASE[offset_based_string],ITSRDF.taClassRef,SDO[schema_type]))
                    else:
                        schema_type = 'Thing'
                        g.add((BASE[offset_based_string],ITSRDF.taClassRef,SDO[schema_type]))    
        # Mention
        else:
            link_probability = annotation['link_probability']
            g.add((BASE[offset_based_string],RDF.type,NIF.EntityOccurrence))
            g.add((BASE[offset_based_string],NIF.entityOccurrenceConf,Literal(link_probability,datatype=XSD.decimal)))
            for ner in ner_result[uid]:
                if ner['spot'] == spot:
                    spacy_type = ner['type']
                    schema_type = translation[spacy_type]
                    g.add((BASE[offset_based_string],ITSRDF.taClassRef,SDO[schema_type]))
                else:
                    schema_type = 'Thing'
                    g.add((BASE[offset_based_string],ITSRDF.taClassRef,SDO[schema_type]))        

In [79]:
print(g.serialize(format="turtle").decode("utf-8"))

@prefix : <http://www.kde.cs.tsukuba.ac.jp/~aso/trec#> .
@prefix itsrdf: <https://www.w3.org/2005/11/its/rdf#> .
@prefix nif: <http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#> .
@prefix schema: <https://schema.org/> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:40113E7E_20229_1133A43_localhost schema:mentions <http://www.kde.cs.tsukuba.ac.jp/~aso/trec#40113E7E_20229_1133A43_localhost#offset_114_123>,
        <http://www.kde.cs.tsukuba.ac.jp/~aso/trec#40113E7E_20229_1133A43_localhost#offset_156_160>,
        <http://www.kde.cs.tsukuba.ac.jp/~aso/trec#40113E7E_20229_1133A43_localhost#offset_177_182>,
        <http://www.kde.cs.tsukuba.ac.jp/~aso/trec#40113E7E_20229_1133A43_localhost#offset_237_242>,
        <http://www.kde.cs.tsukuba.ac.jp/~aso/trec#40113E7E_20229_1133A43_localhost#offset_277_281>,
        <http://www.kde.cs.tsukuba.ac.jp/~aso/trec#40113E7E_20229_1133A43_localhost#offset_342_347>,
        <http://ww